# The start


In [53]:
USE master;
GO
DROP DATABASE IF EXISTS alt;
GO
CREATE DATABASE alt;
GO
USE alt;
GO
CREATE SCHEMA alt;
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:02.502

First table, no many things to change, just be sure that the
version correspond to the format used until now. IF this should 
ever change... just drop/adapt it. but until that time let's play safe
- make use of explicit types
- meaningfull name version could be relevant somewhere else
- CHECK for possible format

In [56]:
-- use alt;
-- GO
CREATE type alt._icdo_version from int not null;
    go

CREATE TABLE alt.icdo_version
(
    icdo_version alt._icdo_version PRIMARY KEY CLUSTERED
        check (icdo_version BETWEEN 1970 and 3000)
    -- 
    -- ... more column on the meaning or meta info on the version
    -- 
);
    GO
---------------------------------------------
insert into alt.icdo_version
values
    (2011),
    (2017);
        GO

Commands completed successfully.

Commands completed successfully.

(2 rows affected)

Total execution time: 00:00:00.072

Ready, defend your self!

In [57]:
-- use alt;
-- PASS: unique version
insert into alt.icdo_version
values
    (2011);
-- PASS: no null
insert into alt.icdo_version
values
    (NULL);
-- PASS: version named after years
insert into alt.icdo_version
values
    (42);
            go

: Msg 2627, Level 14, State 1, Line 3
Violation of PRIMARY KEY constraint 'PK__icdo_ver__479EFC8FA2643FDE'. Cannot insert duplicate key in object 'alt.icdo_version'. The duplicate key value is (2011).

The statement has been terminated.

: Msg 515, Level 16, State 2, Line 7
Cannot insert the value NULL into column 'icdo_version', table 'alt.alt.icdo_version'; column does not allow nulls. INSERT fails.

The statement has been terminated.

: Msg 547, Level 16, State 0, Line 11
The INSERT statement conflicted with the CHECK constraint "CK__icdo_vers__icdo___36B12243". The conflict occurred in database "alt", table "alt.icdo_version", column 'icdo_version'.

The statement has been terminated.

Total execution time: 00:00:00.012

This one is quite blund, but given the way we query this table
we really rely on text, so away with the id... and less naming issue.
- Clear name, because it could also be an icdo_morphologie code and they do not look  the same
- Again use of type, and by default not empty
- No more surrogate 'id' ... I admit that I may in the future reconsider this one
  maybe it is a good idea to rely on ligth surrogate value
... second table and no id!

In [58]:
create type alt._icdo_location_code from varchar(8) NOT NULL;
GO

CREATE TABLE alt.icdo_location
(
    -- -- get the id away from here
    -- id   INT IDENTITY  NOT NULL PRIMARY KEY CLUSTERED,
    icdo_location_code alt._icdo_location_code
        PRIMARY KEY CLUSTERED
        CHECK (icdo_location_code like '[CD][0-9][0-9]%')
    -- later to be used in a varchar(8)
    -- but here it contains 2 codes and one of them
);
GO
insert into alt.icdo_location
    (icdo_location_code)
values
    ('C11'),
    ('C22.2'),
    ('C99.4');
GO

Commands completed successfully.

Commands completed successfully.

(3 rows affected)

Total execution time: 00:00:00.063

Ready, hold on!

In [59]:
-- PASS: Unique
insert into alt.icdo_location
    (icdo_location_code)
select top 1
    icdo_location_code
from alt.icdo_location
order by icdo_location_code asc;
-- PASS: NON NULL
insert into alt.icdo_location
    (icdo_location_code)
select NULL;
-- PASS: Well formatted '[CD][0-9][0-9]%'
insert into alt.icdo_location
    (icdo_location_code)
select 'bad format';
-- PASS: Format ok
insert into alt.icdo_location
    (icdo_location_code)
select 'badfmt';

: Msg 2627, Level 14, State 1, Line 2
Violation of PRIMARY KEY constraint 'PK__icdo_loc__640F8DB339E53C22'. Cannot insert duplicate key in object 'alt.icdo_location'. The duplicate key value is (C11).

The statement has been terminated.

: Msg 515, Level 16, State 2, Line 9
Cannot insert the value NULL into column 'icdo_location_code', table 'alt.alt.icdo_location'; column does not allow nulls. INSERT fails.

The statement has been terminated.

: Msg 8152, Level 16, State 30, Line 13
String or binary data would be truncated.

The statement has been terminated.

: Msg 547, Level 16, State 0, Line 17
The INSERT statement conflicted with the CHECK constraint "CK__icdo_loca__icdo___398D8EEE". The conflict occurred in database "alt", table "alt.icdo_location", column 'icdo_location_code'.

The statement has been terminated.

Total execution time: 00:00:00.010

This one is quite satisfactory. 
- DROP! we have so many values that it is not necessary to have
  a 'text' building block. The text value is a function of code*version
  skip and move!
- no more stupid id name... hurray
- less things to check... less error possible
... na to be honest they we be moved to an other place

In [46]:
/*
--    CREATE TABLE alt.icdo_value (
--        id    INT IDENTITY    NOT NULL PRIMARY KEY CLUSTERED,
--        text  NVARCHAR (400) -- NULL and DUPLICATE allowed?
--        );
--        --------------------------------------------
--        insert into alt.icdo_value(text)
--            values ('C11/2011'), ('C22.2/2011'), ('C11/2017'), ('C99.4/2017');
--            
--            -- PASS ... no table: unique meaning, else what is the point of such a value catalog
--            insert into alt.icdo_value(text) select top 1 text from alt.icdo_value order by id asc;
--            -- PASS ... no table: a code as a meaning
--            insert into alt.icdo_value(text) select NULL;
*/

Commands completed successfully.

Total execution time: 00:00:00.003

- Proper reference for the location and version
- no stupid id...
- appropriate and utterly explicit names

In [47]:
CREATE TABLE alt.icdo_location_version_description
(
    icdo_version alt._icdo_version
        FOREIGN KEY REFERENCES alt.icdo_version(icdo_version)
                ON DELETE NO ACTION ON UPDATE CASCADE,
    icdo_location_code alt._icdo_location_code
        FOREIGN KEY REFERENCES alt.icdo_location(icdo_location_code) 
            ON DELETE NO ACTION ON UPDATE CASCADE,
    -- on delete block. playing safe here
    icdo_location_desc NVARCHAR (200) NOT NULL
        check (icdo_location_desc <> ''),
    -- not null
    -- FOREIGN KEY REFERENCES alt.icdo_value(id) ON DELETE CASCADE ON UPDATE CASCADE,
    -- no more needed! hurray!

    PRIMARY KEY CLUSTERED (icdo_version, icdo_location_code)
    -- Now this get us the value ~ code*version
    -- Functionnal integrity in da place! 
);

insert into alt.icdo_location_version_description
    (icdo_version, icdo_location_code, icdo_location_desc)
select v, c, cast(v as varchar(4)) + '/' + c + '  and blalbaetu'
from (values
        (2011, 'C11') ,
        (2017, 'C11') ,
        (2011, 'C22.2')  ,
        (2017, 'C99.4')) as x(v,c)
select *
from alt.icdo_location_version_description;

(4 rows affected)

(4 rows affected)

Total execution time: 00:00:00.067

icdo_version,icdo_location_code,icdo_location_desc
2011,C11,2011/C11 and blalbaetu
2011,C22.2,2011/C22.2 and blalbaetu
2017,C11,2017/C11 and blalbaetu
2017,C99.4,2017/C99.4 and blalbaetu


Ready, Protect

In [48]:

-- PASS: code * version => text --
insert into alt.icdo_location_version_description
    (icdo_version, icdo_location_code, icdo_location_desc)
select 2011, 'C11', '2011/C11 bis'
-- PASS: 
insert into alt.icdo_location_version_description
    (icdo_version, icdo_location_code, icdo_location_desc)
select 2017, 'C22.2', NULL
-- PASS: No Meaning Rejected
insert into alt.icdo_location_version_description
    (icdo_version, icdo_location_code, icdo_location_desc)
select 2017, 'C22.2', N'          '
insert into alt.icdo_location_version_description
    (icdo_version, icdo_location_code, icdo_location_desc)
select NULL, 'C22.2', N'no version'

: Msg 2627, Level 14, State 1, Line 3
Violation of PRIMARY KEY constraint 'PK__icdo_loc__61DE0454696C6B69'. Cannot insert duplicate key in object 'alt.icdo_location_version_description'. The duplicate key value is (2011, C11).

The statement has been terminated.

: Msg 515, Level 16, State 2, Line 7
Cannot insert the value NULL into column 'icdo_location_desc', table 'alt.alt.icdo_location_version_description'; column does not allow nulls. INSERT fails.

The statement has been terminated.

: Msg 547, Level 16, State 0, Line 11
The INSERT statement conflicted with the CHECK constraint "CK__icdo_loca__icdo___3E52440B". The conflict occurred in database "alt", table "alt.icdo_location_version_description", column 'icdo_location_desc'.

The statement has been terminated.

: Msg 515, Level 16, State 2, Line 14
Cannot insert the value NULL into column 'icdo_version', table 'alt.alt.icdo_location_version_description'; column does not allow nulls. INSERT fails.

The statement has been terminated.

Total execution time: 00:00:00.022

Now to the dungeon

In [49]:
CREATE TABLE alt.diagnose
(
    diagnose_id INT IDENTITY NOT NULL PRIMARY KEY CLUSTERED,
    -- we 
    patient_id INT NOT NULL,
    -- FK/Ref Table removed for clarity
    icd10_code VARCHAR (8)    ,
    icdo_version alt._icdo_version,
    icdo_location_code alt._icdo_location_code
    -- now good! code and use are varchar(8)
    -- hey... anyone ... I'm looking for FK here... no?
);
GO

insert into alt.diagnose
    (patient_id, icd10_code, icdo_location_code, icdo_version)
select 99, 'd44', ichv.icdo_location_code , ichv.icdo_version
from alt.icdo_location_version_description ichv
select *
from alt.diagnose; 
GO  


Commands completed successfully.

(4 rows affected)

(4 rows affected)

Total execution time: 00:00:00.048

diagnose_id,patient_id,icd10_code,icdo_version,icdo_location_code
1,99,d44,2011,C11
2,99,d44,2011,C22.2
3,99,d44,2017,C11
4,99,d44,2017,C99.4


Bring the crone!

In [50]:
ALTER TABLE alt.diagnose 
        ADD FOREIGN KEY (icdo_version, icdo_location_code) 
            REFERENCES alt.icdo_location_version_description(icdo_version, icdo_location_code);     


Commands completed successfully.

Total execution time: 00:00:00.031

Ready, here is the final assault

In [51]:

-- PASS * 3: this can not be unknown at any place
insert into alt.diagnose
    (patient_id, icd10_code, icdo_location_code, icdo_version)
select 99, 'd44', NULL, NULL;
GO

insert into alt.diagnose
    (patient_id, icd10_code, icdo_location_code, icdo_version)
select 99, 'd44', 2011, NULL;
GO

insert into alt.diagnose
    (patient_id, icd10_code, icdo_location_code, icdo_version)
select 99, 'd44', NULL, 'C11';
GO

-- PASS * 3
-- Non allowed combination with non allowed code or value
-- update without where... do not do this at home :)
update alt.diagnose
    set icdo_location_code = 'hell', icdo_version = 666;
GO

select * from alt.diagnose;
GO

-- PASS: unregistered combination on code and version
insert into alt.diagnose
    (patient_id, icd10_code, icdo_location_code, icdo_version)
select 99, 'd44', 'C22.2', 2017;
GO



: Msg 515, Level 16, State 2, Line 3
Cannot insert the value NULL into column 'icdo_location_code', table 'alt.alt.diagnose'; column does not allow nulls. INSERT fails.

The statement has been terminated.

: Msg 515, Level 16, State 2, Line 8
Cannot insert the value NULL into column 'icdo_version', table 'alt.alt.diagnose'; column does not allow nulls. INSERT fails.

The statement has been terminated.

: Msg 515, Level 16, State 2, Line 13
Cannot insert the value NULL into column 'icdo_location_code', table 'alt.alt.diagnose'; column does not allow nulls. INSERT fails.

...

(4 rows affected)

The statement has been terminated.

Total execution time: 00:00:01.043

diagnose_id,patient_id,icd10_code,icdo_version,icdo_location_code
1,99,d44,2011,C11
2,99,d44,2011,C22.2
3,99,d44,2017,C11
4,99,d44,2017,C99.4


: Msg 547, Level 16, State 0, Line 29
The INSERT statement conflicted with the FOREIGN KEY constraint "FK__diagnose__412EB0B6". The conflict occurred in database "alt", table "alt.icdo_location_version_description".

Hurray, all the attack were blocked! No one came through. Here is your reward: The Query that set up this war can now be safely delivered!

In [52]:
select 
    d.patient_id, 
    d.icd10_code, d.icdo_location_code, ilvd.icdo_location_desc
from alt.diagnose d
    join alt.icdo_location_version_description as ilvd
        on ilvd.icdo_version = d.icdo_version
        and ilvd.icdo_location_code = d.icdo_location_code

(4 rows affected)

Total execution time: 00:00:00.152

patient_id,icd10_code,icdo_location_code,icdo_location_desc
99,d44,C11,2011/C11 and blalbaetu
99,d44,C22.2,2011/C22.2 and blalbaetu
99,d44,C11,2017/C11 and blalbaetu
99,d44,C99.4,2017/C99.4 and blalbaetu
